In [5]:
import json
import time
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add the parent directory (which contains 'office') to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from office.delivery_env import DeliveryRobotEnv


pygame 2.6.1 (SDL 2.28.4, Python 3.10.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:



def replay_episode(json_path, render_delay=0.1):
    # Load the saved trajectory
    with open(json_path, 'r') as f:
        transitions = json.load(f)

    # Initialize the environment in human render mode
    env = DeliveryRobotEnv(
        config="open_office_simple",
        show_walls=True,
        show_obstacles=True,
        show_carpets=False,
        render_mode="human"
    )

    obs, _ = env.reset()

    for step in transitions:
        action = step["action"]

        # Take action in environment
        obs, reward, terminated, truncated, _ = env.step(action)

        # Render frame
        env.render()

        # Delay between frames for visibility
        time.sleep(render_delay)

        if terminated or truncated:
            break

    env.close()

# Example usage
replay_episode("logs/episode_0.json", render_delay=0.05)


pygame 2.6.1 (SDL 2.28.4, Python 3.10.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets
from IPython.display import display

from office.delivery_env import DeliveryRobotEnv


def draw_trajectory(json_path, use_next_state=False):
    # Load transitions
    with open(json_path, 'r') as f:
        transitions = json.load(f)

    key = "next_state" if use_next_state else "state"
    positions = np.array([step[key][:2] for step in transitions])
    positions[:, 0] *= 800
    positions[:, 1] *= 600
    num_steps = len(positions)
    alphas = np.linspace(0.2, 1.0, num_steps)

    # Load env (for static features)
    env = DeliveryRobotEnv(
        config="open_office_simple",
        show_walls=True,
        show_obstacles=True,
        show_carpets=False,
        render_mode="human"
    )

    # Setup plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_facecolor((55/255, 55/255, 55/255))
    ax.set_xlim(0, 800)
    ax.set_ylim(0, 600)
    ax.invert_yaxis()
    ax.set_aspect('equal')
    ax.set_title("Trajectory (Interactive Slider)")

    # --- Draw static features once ---
    for wall in env.walls:
        ax.add_patch(patches.Rectangle((wall[0], wall[1]), wall[2], wall[3], facecolor='white'))

    for obs in env.obstacles:
        ax.add_patch(patches.Circle((obs[0], obs[1]), obs[2],
                                    facecolor=(1, 0.6, 0.6), edgecolor='red', linewidth=2))

    for i, table in enumerate(env.tables):
        tx, ty, tw, th = table
        is_delivered = i in env.delivered_tables
        color = 'green' if is_delivered else 'orange'
        ax.add_patch(patches.Rectangle((tx, ty), tw, th, facecolor=color, edgecolor='black', linewidth=2))
        if not is_delivered:
            margin = env.robot_radius * 4
            ax.add_patch(patches.Rectangle(
                (tx - margin, ty - margin), tw + 2 * margin, th + 2 * margin,
                facecolor=(0, 1, 0, 0.25), linewidth=0
            ))

    # --- Pre-allocate trajectory segments ---
    lines = []
    for i in range(1, num_steps):
        line, = ax.plot([], [], color='blue', alpha=alphas[i], linewidth=2)
        lines.append(line)

    # --- Robot dot ---
    robot_dot = patches.Circle((positions[0][0], positions[0][1]), radius=6, color=(0, 100/255, 1))
    ax.add_patch(robot_dot)

    # --- Slider ---
    slider = widgets.IntSlider(
        value=num_steps - 1, min=1, max=num_steps - 1, step=1,
        description="Step", continuous_update=True
    )

    def update(step):
        # Update lines
        for i, line in enumerate(lines):
            if i < step:
                x_vals = positions[i:i+2, 0]
                y_vals = positions[i:i+2, 1]
                line.set_data(x_vals, y_vals)
            else:
                line.set_data([], [])

        # Update robot dot
        robot_dot.center = (positions[step][0], positions[step][1])
        fig.canvas.draw_idle()

    slider.observe(lambda change: update(change['new']), names='value')

    # Initial render
    display(slider)
    update(num_steps - 1)
    plt.show()

    env.close()


In [ ]:
draw_trajectory("../logs/episode_0.json")


IntSlider(value=1816, continuous_update=False, description='Step:', max=1816, min=1)

Output()